In [3]:
import pandas, geopandas, numpy, os

folder = "D:/Softwares/OneDrive - Amdev Property/3 School of Cities/Census Data Analysis/Data"

GGH_Census_06 = pandas.read_csv("../census-summary-data-2006-2021/part-c-results/GGH-Census-da-06.csv")
GGH_Census_21 = pandas.read_csv("../census-summary-data-2006-2021/part-c-results/GGH-Census-da-21.csv")

GGH_DA_with_CSDUID_06 = pandas.read_csv("../census-summary-data-2006-2021/part-b-results/GGH-boundaries-intersect-06.csv")
GGH_DA_with_CSDUID_21 = pandas.read_csv("../census-summary-data-2006-2021/part-b-results/GGH-boundaries-intersect-21.csv")

region_summary = pandas.read_csv("../regions-summary.csv")

GGH_Census_06 = GGH_Census_06.drop("Unnamed: 0", axis =1)
GGH_Census_21 = GGH_Census_21.drop("Unnamed: 0", axis =1)

In [22]:
# This step is to just create a cleaner version of GGH_DA_with_CSDUID and join with census data
# PerAreaUGC is the percentage of area of a DA within the UGC, same idea for PerAreablt, PerAreatbf
# BLT is built boundary not a sandwich 
# TBF is transit buffer (1km)
GGH_DA_with_CSDUID_06_concise = GGH_DA_with_CSDUID_06[['DAUID', 'PerAreaBLT', 'PerAreaUGC', 'PerAreaTBF']]
GGH_DA_with_CSDUID_06_concise['DAUID'] = GGH_DA_with_CSDUID_06_concise['DAUID'].astype('str')
GGH_Census_06['DAUID'] = GGH_Census_06['DAUID'].astype('str')

GGH_DA_with_CSDUID_06_concat = pandas.merge(GGH_Census_06, GGH_DA_with_CSDUID_06_concise, on = 'DAUID')
GGH_DA_with_CSDUID_06_concat = GGH_DA_with_CSDUID_06_concat.replace(numpy.nan, 0)

GGH_DA_with_CSDUID_21_concise = GGH_DA_with_CSDUID_21[['DAUID', 'PerAreaBLT', 'PerAreaUGC', 'PerAreaTBF']]
GGH_DA_with_CSDUID_21_concise['DAUID'] = GGH_DA_with_CSDUID_21_concise['DAUID'].astype('str')
GGH_Census_21['DAUID'] = GGH_Census_21['DAUID'].astype('str')

GGH_DA_with_CSDUID_21_concat = pandas.merge(GGH_Census_21, GGH_DA_with_CSDUID_21_concise, on = 'DAUID')
GGH_DA_with_CSDUID_21_concat = GGH_DA_with_CSDUID_21_concat.replace(numpy.nan, 0)

#loop through the census fields to create individual table of each census variables. 
#calculating using spatial interpolation, assuming population is evernly distributed within the Dissemination Area
#weight the census variable by area. 

i = 4
dataframeList = []
fieldList = len(GGH_DA_with_CSDUID_21_concat.columns.tolist()[4:12])*2
while i < fieldList:
    
    fields = [GGH_DA_with_CSDUID_21_concat, GGH_DA_with_CSDUID_06_concat] 
    for field in fields:
        #area interpolating the census variables. 
        year = field.columns.tolist()[i].split("-")[0]
        
        field['PerAreaBLT'] = field['PerAreaBLT'] .astype('float')
        field['PerAreaUGC'] = field['PerAreaUGC'] .astype('float')
        field['PerAreaTBF'] = field['PerAreaTBF'] .astype('float')
        
        #calculate portion of the census variable within the three geographical area. 
        frame = field[['DAUID', 'Municipality',  field.columns.tolist()[i]] ] #, 'PerAreaBLT', 'PerAreaUGC', 'PerAreaTBF'
        frame[f'{year}-1-Portion in Built Boundary'] = field[field.columns.tolist()[i]]*field['PerAreaBLT'] 
        frame[f'{year}-2-Portion in Urban Gowth Centre'] = field[field.columns.tolist()[i]]*field['PerAreaUGC']
        frame[f'{year}-3-Portion in Transit Buffer'] = field[field.columns.tolist()[i]]*field['PerAreaTBF']
        
        frameFields = frame.columns.tolist()[2:]
        ## PIVOOOTT table
        #pivot table to dissolve by municipalities to calculate the total of each census variable within the municipalities. 
        frame = pandas.pivot_table(data = frame, values = frameFields, index = 'Municipality', aggfunc = numpy.sum)
        frame = frame.reset_index()
        dataframeList.append(frame)
    
    i+=1

#merge the pivot tables (merge 2006 with 2021)
mergedTable_list = []
i = 0
while i < len(dataframeList):
    merge_table = pandas.merge(dataframeList[i], dataframeList[i+1], on = "Municipality", sort = False)
    
    #calculate changes between 2006 and 2021
    a=1
    while a <= len(merge_table.columns.tolist()[1:5]):
        changeName = "-".join(merge_table.columns.tolist()[a].split("-")[1:])
        merge_table[f'Chg-{changeName}'] = merge_table[merge_table.columns.tolist()[a]] - merge_table[merge_table.columns.tolist()[a+4]]
        a+=1
    mergedTable_list.append(merge_table)
    i+=2
    
#calculate the percent of changes in each city that happened within each geographic area
#i.e. City A grew by 100 person overall, Built Area A (within City A) grew by 50 people, then 50% of the growth happened in Built Area A. 
#i.e. City B grew by 50 person overall, Built Area B (within City B) grew by 200 people, then 400% of the growth happened in Built Area B. 
#City B scenario is possible because areas outside of Built Area B can experience decline while Built Area B still experience growth. 
for merged in mergedTable_list:
    fields = merged.columns.tolist()[1:]
    name = fields[3].split("-")[2]
    print(name)
    
    merged[f'% of {name} Chg In Built Boundary'] = round((merged[fields[8]] / merged[fields[11]])*100,2)
    merged[f'% of {name} Chg In UGC'] = round((merged[fields[9]] / merged[fields[11]])*100,2)
    merged[f'% of {name} Chg In Transit Buffer'] = round((merged[fields[10]] / merged[fields[11]])*100,2)

    #renaming columns

    i = 1 
    newFields = merged.columns.tolist()[:13]
    while i < len(newFields):
        title = newFields[i].split("-")[0]
        fieldName = newFields[i].split("-")[2]
        merged = merged.rename(columns={f"{newFields[i]}": f"{title} {fieldName}"})
        
        i+=1
    

C:\Users\Michael Liu\AppData\Local\Temp\ipykernel_11592\1619636493.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GGH_DA_with_CSDUID_06_concise['DAUID'] = GGH_DA_with_CSDUID_06_concise['DAUID'].astype('str')
C:\Users\Michael Liu\AppData\Local\Temp\ipykernel_11592\1619636493.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GGH_DA_with_CSDUID_21_concise['DAUID'] = GGH_DA_with_CSDUID_21_concise['DAUID'].astype('str')
C:\Users\Michael Liu\AppData\Local\Temp\ipykernel_11592\1619636493.py:39: SettingWith

Population
Total dwellings
Total dwelling type
Single detached house
Semi detached house
Row house
Apartment or flat duplex
Apartment five or more storeys
Apartment fewer than five storeys
Other single attached house
Movable dwelling
Total households


C:\Users\Michael Liu\AppData\Local\Temp\ipykernel_11592\1619636493.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[f'{year}-1-Portion in Built Boundary'] = field[field.columns.tolist()[i]]*field['PerAreaBLT']
C:\Users\Michael Liu\AppData\Local\Temp\ipykernel_11592\1619636493.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame[f'{year}-2-Portion in Urban Gowth Centre'] = field[field.columns.tolist()[i]]*field['PerAreaUGC']
C:\Users\Michael Liu\AppData\Local\Temp\ipykernel_11592\1619636493.p

In [23]:
region_summary_list = []
i = 0 
while i < len(mergedTable_list):
    #we loop through the mergedTable_list to calculate the total and gradnd total 
    #
    region_summary = region_summary[['Municipality', 'Macro']]
    mergedTable = merged['Municipality'].reset_index().drop('index', axis =1)
    mergedTable[merged.columns.tolist()[9:13]] = merged[merged.columns.tolist()[9:13]].copy()
    region_summary = pandas.merge(mergedTable_list[i], region_summary, on = "Municipality")

    #Calculate GGH and Outer Ring Totals
    #using the pivot table to only sum the fields withou percentages
    ggh_summary = pandas.pivot_table(region_summary, values = region_summary.columns.tolist()[9:13], index = 'Macro', aggfunc = numpy.sum).reset_index()

    #calculate the ratio between the geographical areas to the regions, 
    ggh_summary['% of Chg In Built Boundary'] = round(ggh_summary[ggh_summary.columns.tolist()[1]] / ggh_summary[ggh_summary.columns.tolist()[4]]*100,2)
    ggh_summary['% of Chg In UGC'] = round(ggh_summary[ggh_summary.columns.tolist()[2]] / ggh_summary[ggh_summary.columns.tolist()[4]]*100,2)
    ggh_summary['% of Chg In Transit Buffer'] = round(ggh_summary[ggh_summary.columns.tolist()[3]] / ggh_summary[ggh_summary.columns.tolist()[4]]*100,2)

    region_summary = region_summary.drop(region_summary.columns.tolist()[1:9], axis = 1)

    #Calculatinge the Grand Total Row
    total = ["Grand Total"]
    a = 1
    while a < len(ggh_summary.columns.tolist()):
        #get the values for GTHA and Outer Ring
        gtha = ggh_summary[ggh_summary.columns.tolist()[a]].values[0]
        outer = ggh_summary[ggh_summary.columns.tolist()[a]].values[1]
        if a <= 4:
            #summing the change
            total.append(round(gtha + outer,2))
        else:
            #Calculate percentage change using the previous 4 values
            total.append(round(total[a-4]/total[4]*100,2)) 
        a+=1
    ggh_summary.loc[len(ggh_summary)] = total

    #Organizing the Exporting Table
    region_summarys = region_summary.sort_values(by = ["Macro", "Municipality"]).reset_index().drop('index', axis = 1)
    #region_summarys = region_summarys.drop(region_summarys.columns.tolist()[1:9], axis = 1)

    #Add Labels to Macro
    gtha_values = ggh_summary.values.tolist()[0]
    gtha_values.append("GTHA SubTotal")
    gtha_values[0] = "GTHA SubTotal"

    out_value = ggh_summary.values.tolist()[1]
    out_value.append("Outer Ring SubTotal")
    out_value[0] = "Outer Ring SubTotal"

    total_value = ggh_summary.values.tolist()[2]
    total_value.append("Grand Total")
    total_value[0] = "Grand Total"

    #Insert the rows
    region_summarys.loc[6] = gtha_values
    region_summarys.loc[21] = out_value
    region_summarys.loc[22] = total_value

    #Shifting the location of "Macro" field
    macro = region_summarys["Macro"]
    region_summarys = region_summarys.drop("Macro", axis = 1)
    region_summarys.insert(0, "Macro", macro)

    region_summarys = region_summarys.drop(region_summarys.columns.tolist()[2:5], axis = 1)
    fieldName = ggh_summary.columns.tolist()[4].split("-")[2]
    region_summarys = region_summarys.rename(columns={f"{ggh_summary.columns.tolist()[4]}": f"Changes in {fieldName}"})

    #export to csv
    #region_summarys.to_csv(f"../census-summary-data-2006-2021/part-c-results/summarizing-results/{fieldName}.csv")

    region_summary_list.append(region_summarys)

    
    i+=1

In [11]:
gtha_values

[' ',
 358251.9313144408,
 112710.21133748312,
 138836.05181124478,
 521460.0,
 68.7,
 21.61,
 26.62,
 'GTHA SubTotal']

In [24]:
table = region_summary_list[0]
for region in region_summary_list[1:]:
    region = region.drop("Macro", axis =1)
    table = pandas.merge(table, region, on = "Municipality")

table.to_csv(f"../census-summary-data-2006-2021/part-c-results/summarizing-results/table.csv")
    


In [25]:
table

,Macro,Municipality,Changes in Population,% of Population Chg In Built Boundary,% of Population Chg In UGC,% of Population Chg In Transit Buffer,Changes in Total dwellings,% of Total dwellings Chg In Built Boundary,% of Total dwellings Chg In UGC,% of Total dwellings Chg In Transit Buffer,...,% of Other single attached house Chg In UGC,% of Other single attached house Chg In Transit Buffer,Changes in Movable dwelling,% of Movable dwelling Chg In Built Boundary,% of Movable dwelling Chg In UGC,% of Movable dwelling Chg In Transit Buffer,Changes in Total households,% of Total households Chg In Built Boundary,% of Total households Chg In UGC,% of Total households Chg In Transit Buffer
0,GTHA,City of Hamilton,66456.0,43.62,3.24,4.84,28818.0,55.88,3.98,6.88,...,7.35,19.88,-140.0,46.49,-3.57,0.03,28805.0,56.48,4.03,6.30
1,GTHA,City of Toronto,301593.0,103.05,52.98,65.72,215824.0,101.10,55.15,64.57,...,25.87,45.24,-190.0,99.98,16.11,50.51,184980.0,101.24,52.44,62.15
2,GTHA,Region of Durham,135752.0,57.92,0.90,1.96,48414.0,65.99,1.75,3.24,...,-6.69,-0.74,-75.0,83.85,-0.00,2.33,48390.0,65.94,1.73,3.15
3,GTHA,Region of Halton,157381.0,36.52,1.28,4.54,51976.0,43.51,2.85,6.41,...,-1.19,43.44,-40.0,12.83,0.01,-5.88,51625.0,44.76,3.01,6.24
4,GTHA,Region of Peel,292975.0,39.49,6.80,6.06,94515.0,53.46,10.34,7.18,...,6.37,15.32,105.0,97.38,-5.26,-34.97,92090.0,53.40,10.60,7.07
5,GTHA,Region of York,280975.0,27.57,1.51,8.02,119092.0,43.69,1.96,9.43,...,1.86,62.38,-25.0,185.85,-0.00,27.54,115570.0,43.71,2.07,9.35
6,GTHA SubTotal,GTHA SubTotal,1235132.0,54.17,15.33,20.36,558639.0,70.07,24.09,29.40,...,39.17,70.46,-365.0,73.23,8.53,38.09,521460.0,68.70,21.61,26.62
7,Outer Ring,City of Brantford,14496.0,34.00,4.26,0.00,6306.0,53.21,5.09,0.00,...,43.17,-0.00,0.0,inf,NaN,NaN,6085.0,53.15,4.39,0.00
8,Outer Ring,City of Guelph,28797.0,59.18,4.05,5.06,11777.0,62.51,5.20,5.87,...,inf,inf,-15.0,73.31,-0.00,-0.00,11805.0,63.95,5.43,6.86
9,Outer Ring,City of Kawartha Lakes,4686.0,36.46,0.00,0.00,961.0,121.40,0.00,0.00,...,-0.00,-0.00,100.0,-8.24,0.00,0.00,3235.0,36.34,0.00,0.00


In [ ]:
"""i = 1
#merging all the Percentage Fields into one table. 
per_chg_field = mergedTable_list[0].columns.tolist()[-3:]
municipality = mergedTable_list[0].columns.tolist()[0]
joinFrame = mergedTable_list[0][municipality].reset_index()
joinFrame = joinFrame.drop("index", axis = 1)

for f in per_chg_field:
    joinFrame[f] = mergedTable_list[0][f]
while i < len(mergedTable_list):
    per_chg_field = mergedTable_list[i].columns.tolist()[-3:]
    municipality = mergedTable_list[i].columns.tolist()[0]
    joiningFrame = mergedTable_list[i][municipality].reset_index()
    joiningFrame = joiningFrame.drop("index", axis = 1)

    for f in per_chg_field:
        joiningFrame[f] = mergedTable_list[i][f]

    joinFrame = pandas.merge(joinFrame,  joiningFrame, on = "Municipality")
    i+=1

ggh_regions = geopandas.read_file("../ggh-growth-plan-regions.geojson")

ggh1 = pandas.merge(ggh_regions, joinFrame, right_on = "Municipality", left_on = "Name", sort=False).drop(['Name'], axis = 1)
ggh1.explore(column = "Municipality", legend = True, cmap = "Purples", scheme="quantiles")"""

'i = 1\n#merging all the Percentage Fields into one table. \nper_chg_field = mergedTable_list[0].columns.tolist()[-3:]\nmunicipality = mergedTable_list[0].columns.tolist()[0]\njoinFrame = mergedTable_list[0][municipality].reset_index()\njoinFrame = joinFrame.drop("index", axis = 1)\n\nfor f in per_chg_field:\n    joinFrame[f] = mergedTable_list[0][f]\nwhile i < len(mergedTable_list):\n    per_chg_field = mergedTable_list[i].columns.tolist()[-3:]\n    municipality = mergedTable_list[i].columns.tolist()[0]\n    joiningFrame = mergedTable_list[i][municipality].reset_index()\n    joiningFrame = joiningFrame.drop("index", axis = 1)\n\n    for f in per_chg_field:\n        joiningFrame[f] = mergedTable_list[i][f]\n\n    joinFrame = pandas.merge(joinFrame,  joiningFrame, on = "Municipality")\n    i+=1\n\nggh_regions = geopandas.read_file("../ggh-growth-plan-regions.geojson")\n\nggh1 = pandas.merge(ggh_regions, joinFrame, right_on = "Municipality", left_on = "Name", sort=False).drop([\'Name\'],